### Pobranie danych

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime as dt, timedelta as tmd
import tqdm

from importlib import reload
import DataLoader.xAPIConnector
reload(DataLoader.xAPIConnector)
from DataLoader.xAPIConnector import *


import DataLoader.DataLoader
reload(DataLoader.DataLoader)
from DataLoader.DataLoader import *

# import xAPIConnector.config
# reload(xAPIConnector.config)
# from xAPIConnector.config import user_id, pwd

user_id = "17340399"
pwd = "MASPROJEKT2025"

In [ ]:
symbols = ['BITCOIN', 'ETHEREUM']
start, interval = '2024-12-01 00:00:00', '5min'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

In [ ]:
data=data.reset_index()
data['ratio'] = data['BITCOIN'] / data['ETHEREUM']

### Relative Strength Index (RSI)

In [ ]:
def calculate_RSI(data,col,dl,i):
    x=data[col][i:i+dl+1]
    x=x.diff()[1:]
    gorna_sr=0
    dolna_sr=0
    for y in x:
        if y>0:
            gorna_sr+=y
        else:
            dolna_sr-=y
    rs=gorna_sr/dolna_sr
    rsi=100*(1-1/(1+rs))
    return rsi

In [ ]:
def RSI_stack(data,col,dl=14,gorna_gr=70,dolna_gr=30):
    
    data['position'] = 0 
    data['entry_price'] = np.nan
    data['pnl'] = 0.0  
    data['trade_pnl'] = 0.0  
    data['RSI']=0.0
    cumulative_pnl=0.0
    
    for i in range(dl,len(data)):
        price = data.loc[i, col]
        rsi=calculate_RSI(data,col,dl,i-dl)
        data.loc[i,'RSI'] = rsi

        if data.loc[i - 1, 'position'] == 0:  
            
            if rsi > gorna_gr:
                data.loc[i, 'position'] = -1
                data.loc[i, 'entry_price'] = price
            elif rsi < dolna_gr:
                data.loc[i, 'position'] = 1
                data.loc[i, 'entry_price'] = price
        
        else:
            entry_price = data.loc[i - 1, 'entry_price']
            position = data.loc[i - 1, 'position']

            trade_pnl = (price - entry_price) * position
            data.loc[i, 'trade_pnl'] = trade_pnl

            if position == 1 and rsi > gorna_gr:
                data.loc[i, 'position'] = 0  
                cumulative_pnl += trade_pnl
            elif position == -1 and rsi < dolna_gr:
                data.loc[i, 'position'] = 0  
                cumulative_pnl += trade_pnl
            else:
                data.loc[i, 'position'] = position
                data.loc[i, 'entry_price'] = entry_price
        data.loc[i, 'pnl'] = cumulative_pnl

In [ ]:
import pandas as pd
import numpy as np

def mean_revert_with_pnl(data, col, roll=10, tp=1, sl=3, entry=2):
    data = data.reset_index(drop=True)
    
    data['rolling_mean'] = data[col].rolling(window=roll).mean()
    data['rolling_std'] = data[col].rolling(window=roll).std()

    data['position'] = 0 
    data['entry_price'] = np.nan  
    data['exit_reason'] = '' 
    data['pnl'] = 0.0  
    data['trade_pnl'] = 0.0  

    cumulative_pnl = 0.0 

    for i in range(roll, len(data)):
        mean = data.loc[i, 'rolling_mean']
        std = data.loc[i, 'rolling_std']
        price = data.loc[i, col]

        if pd.isna(mean) or pd.isna(std):
            continue

        if data.loc[i - 1, 'position'] == 0:  
            
            if price > mean + std * entry:
                data.loc[i, 'position'] = -1
                data.loc[i, 'entry_price'] = price
            elif price < mean - std * entry:
                data.loc[i, 'position'] = 1
                data.loc[i, 'entry_price'] = price
        else:
            entry_price = data.loc[i - 1, 'entry_price']
            position = data.loc[i - 1, 'position']

            trade_pnl = (price - entry_price) * position
            data.loc[i, 'trade_pnl'] = trade_pnl

            if position == 1 and price > mean + std * sl:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'sl'
                cumulative_pnl += trade_pnl
            elif position == -1 and price < mean - std * sl:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'sl'
                cumulative_pnl += trade_pnl

            elif position == 1 and price > mean - std * tp:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'tp'
                cumulative_pnl += trade_pnl
            elif position == -1 and price < mean + std * tp:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'tp'
                cumulative_pnl += trade_pnl

            elif (position == 1 and price >= mean) or (position == -1 and price <= mean):
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'mean'
                cumulative_pnl += trade_pnl

            else:
                data.loc[i, 'position'] = position
                data.loc[i, 'entry_price'] = entry_price

        data.loc[i, 'pnl'] = cumulative_pnl

    return data

In [ ]:
data3=mean_revert_with_pnl(data, "ratio",10, 0.5, 1.5, 1)
plt.plot(data3['pnl'])

In [ ]:
import pandas as pd
import numpy as np

def mean_revert_with_RSI(data, col,roll=10, tp=1, sl=3, entry=2, dl=14,gorna_gr=70,dolna_gr=30):
    data = data.reset_index(drop=True)
    
    data['rolling_mean'] = data[col].rolling(window=roll).mean()
    data['rolling_std'] = data[col].rolling(window=roll).std()

    data['position'] = 0 
    data['entry_price'] = np.nan  
    data['exit_reason'] = '' 
    data['pnl'] = 0.0  
    data['trade_pnl'] = 0.0  

    cumulative_pnl = 0.0 

    for i in range(max(roll,dl), len(data)):
        mean = data.loc[i, 'rolling_mean']
        std = data.loc[i, 'rolling_std']
        rsi=calculate_RSI(data,col,dl,i-dl)
        data.loc[i,'RSI'] = rsi
        price = data.loc[i, col]

        if pd.isna(mean) or pd.isna(std):
            continue

        if data.loc[i - 1, 'position'] == 0:  
            
            if price > mean + std * entry:
                data.loc[i, 'position'] = -1
                data.loc[i, 'entry_price'] = price
            elif price < mean - std * entry:
                data.loc[i, 'position'] = 1
                data.loc[i, 'entry_price'] = price
            if rsi > gorna_gr:
                data.loc[i, 'position'] = -1
                data.loc[i, 'entry_price'] = price
            elif rsi < dolna_gr:
                data.loc[i, 'position'] = 1
                data.loc[i, 'entry_price'] = price
        
        else:
            entry_price = data.loc[i - 1, 'entry_price']
            position = data.loc[i - 1, 'position']

            trade_pnl = (price - entry_price) * position
            data.loc[i, 'trade_pnl'] = trade_pnl

            if position == 1 and price > mean + std * sl:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'sl'
                cumulative_pnl += trade_pnl
            elif position == -1 and price < mean - std * sl:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'sl'
                cumulative_pnl += trade_pnl

            elif position == 1 and price > mean - std * tp:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'tp'
                cumulative_pnl += trade_pnl
            elif position == -1 and price < mean + std * tp:
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'tp'
                cumulative_pnl += trade_pnl

            elif (position == 1 and price >= mean) or (position == -1 and price <= mean):
                data.loc[i, 'position'] = 0  
                data.loc[i, 'exit_reason'] = 'mean'
                cumulative_pnl += trade_pnl
            
            elif position == 1 and rsi > gorna_gr:
                data.loc[i, 'position'] = 0  
                cumulative_pnl += trade_pnl
            elif position == -1 and rsi < dolna_gr:
                data.loc[i, 'position'] = 0  
                cumulative_pnl += trade_pnl

            else:
                data.loc[i, 'position'] = position
                data.loc[i, 'entry_price'] = entry_price

        data.loc[i, 'pnl'] = cumulative_pnl

    return data

In [ ]:
data4=mean_revert_with_RSI(data, "ratio",10, 0.5, 1.5, 1)
plt.plot(data4['pnl'])